Это конвертер keras-модели в модель TensorFlow.

In [3]:
import keras
import keras.models
import numpy as np
from keras.models import model_from_json

SyntaxError: invalid syntax (<ipython-input-3-3fa0248759f6>, line 1)

In [2]:
from keras.utils.generic_utils import CustomObjectScope

In [3]:
name = 'm1' # Здесь укажите имя файла с вашей моделью, вам потребуется .h5 и .json файл (веса и топология)
# load json and create model
json_file = open(name + '.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
# Следующий 'with' нужен, если в вашей модели используется функция активации relu6, если её у вас нет - 'with' можно убрать
with CustomObjectScope({'relu6': keras.layers.ReLU(6.)}):
    loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights(name + ".h5")
print("Loaded model from disk")

W1209 15:20:58.221551 26104 deprecation_wrapper.py:119] From C:\Users\pveryovkin\AppData\Local\Continuum\anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W1209 15:20:58.252835 26104 deprecation_wrapper.py:119] From C:\Users\pveryovkin\AppData\Local\Continuum\anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W1209 15:20:58.280180 26104 deprecation_wrapper.py:119] From C:\Users\pveryovkin\AppData\Local\Continuum\anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W1209 15:20:58.311112 26104 deprecation_wrapper.py:119] From C:\Users\pveryovkin\AppData\Local\Continuum\anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:174: The name tf.get_default_session is deprec

Loaded model from disk


In [4]:
loaded_model.summary() # Выводим информацию о модели

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 256, 256, 1)  0                                            
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 128, 128, 32) 288         input_1[0][0]                    
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 128, 128, 32) 128         conv1[0][0]                      
__________________________________________________________________________________________________
conv1_relu (Activation)         (None, 128, 128, 32) 0           conv1_bn[0][0]                   
__________________________________________________________________________________________________
conv_dw_up

In [5]:
loaded_model.input # Выводим информацию о том, какие данные должны поступать на вход НС

<tf.Tensor 'input_1:0' shape=(?, 256, 256, 1) dtype=float32>

In [6]:
# Нарисуем модель в .png файл (в виде графа)
from keras.utils import plot_model
plot_model(loaded_model, to_file = name + '.png')

In [7]:
loaded_model.output # Выводим информацию о том, какие данные имеем на выходе

<tf.Tensor 'final_activation/truediv:0' shape=(?, 256, 256, 2) dtype=float32>

In [8]:
import tensorflow as tf
from tensorflow.python.framework import graph_util
from tensorflow.python.framework import graph_io

from keras import backend as K
from keras.models import load_model

# Функция экспорта загруженной модели в TensorFlow
def export_keras_to_tf(keras_model, output_model, num_output):
    predictions = [None] * num_output
    predrediction_node_names = [None] * num_output

    for i in range(num_output):
        predrediction_node_names[i] = 'output_node' + str(i)
        predictions[i] = tf.identity(keras_model.outputs[i], name=predrediction_node_names[i])

    sess = K.get_session()

    constant_graph = graph_util.convert_variables_to_constants(sess, sess.graph.as_graph_def(), predrediction_node_names)
    infer_graph = graph_util.remove_training_nodes(constant_graph) 

    graph_io.write_graph(infer_graph, '.', output_model, as_text=False)


In [1]:
# Сконвертируем загруженную модель в формат TensorFlow
num_output = 1
output_model = name + '.pb'
predrediction_node_names = export_keras_to_tf(loaded_model, output_model, num_output)

print('Ouput nodes are:', predrediction_node_names)
print('Saved as TF frozen model to: ', output_model)


NameError: name 'name' is not defined